In [1]:
# see https://docs.astropy.org/en/stable/io/fits/
!python -m pip install "astropy"

You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


In [3]:
from astropy.io import fits
import os,sys
import numpy as np

sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *

filenames=[
    r"D:/bokeh-examples/astropy-arecibo/fits/sum.fits",
    r"D:/bokeh-examples/astropy-arecibo/fits/sum1.fits",
    r"D:/bokeh-examples/astropy-arecibo/fits/sum2.fits",
    r"D:/bokeh-examples/astropy-arecibo/fits/sum3.fits",
    r"D:/bokeh-examples/astropy-arecibo/fits/sum4.fits",
]

# filenames=[
#     r"D:/bokeh-examples/astropy-arecibo/fits/sum1.fits"
# ]

scale=10.0
x_dim = int(400*scale)
y_dim = int(500*scale)

dims=[x_dim,y_dim]

N=len(filenames)

# multifield
bMultiField=False
if bMultiField:
    fields=[Field('field{}'.format(I),'float32','row_major') for I in range(N)]
    db=CreateIdx(url="arecibo.idx", dims=[x_dim,y_dim],fields=fields,compression="raw")
# multitime
else:
    field=Field('field','float32','row_major')
    db=CreateIdx(url="arecibo.big.idx", dims=[x_dim,y_dim],fields=[field],time=[0,N-1,"time_%02d/"],compression="raw")
print(db.getDatasetBody().toString())

<dataset url="arecibo.big.idx" typename="IdxDataset">
	<idxfile>
		<version value="6" />
		<bitmask value="V1010101010101010101010101" />
		<box value="0 4000 0 5000" />
		<bitsperblock value="16" />
		<blocksperfile value="512" />
		<block_interleaving value="0" />
		<filename_template value="./arecibo.big/%04x.bin" />
		<missing_blocks value="False" />
		<time_template value="time_%02d/" />
		<field name="field" description="" index="" default_compression="" default_layout="" default_value="0" filter="" dtype="float32" />
		<timestep from="0" to="5" step="1" />
	</idxfile>
</dataset>


In [4]:
for I,filename in enumerate(filenames):
    print("Converting filename",filename,"...")
    hdul = fits.open(filename)
    print("-----------------------------------------")
    hdul.info()
    print("-----------------------------------------")
    nlayers=len(hdul)
    print("-----------------------------------------")
    Assert(nlayers==1)
    print("-----------------------------------------")
    a=hdul[0].data
    print((np.min(a),np.max(a)) )
    b = np.matrix(a,dtype=numpy.single)
    print((np.min(b),np.max(b)) )
    
    # rescale if needed
    if scale!=1.0:
        from skimage.transform import resize
        b = resize(b, (int(scale*b.shape[0]), int(scale*b.shape[1])))
        
    print("-----------------------------------------")
    db.write(b,
             field=fields[I].name if bMultiField else db.getField().name, 
             time=0 if bMultiField else I
    ) #NOTE BUG HERE if I use fields[I]
    print("-----------------------------------------")
    hdul.close()
    print("-----------------------------------------")
    print("conversion done")
print("All done")

Converting filename D:/bokeh-examples/astropy-arecibo/fits/sum.fits ...
-----------------------------------------
Filename: D:/bokeh-examples/astropy-arecibo/fits/sum.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     143   (400, 500)   float32   
-----------------------------------------
-----------------------------------------
-----------------------------------------
(-3.002467, 4.737127)
(-3.002467, 4.737127)
-----------------------------------------
-----------------------------------------
-----------------------------------------
conversion done
Converting filename D:/bokeh-examples/astropy-arecibo/fits/sum1.fits ...
-----------------------------------------
Filename: D:/bokeh-examples/astropy-arecibo/fits/sum1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     107   (400, 500)   float32   
-----------------------------------------
-----------------------------------------
--

In [17]:
# db.compressDataset(["zip"])